<center><b style="font-size: 200%">Project I</b><br><br><b style="font-size: 150%"> Data PreProcessing - Analyzing borrowers’ risk of defaulting<center>


<b style="font-size: 150%">Description</b>

project I is about prepare a report for a bank’s loan division. detriment whether the customers who applied for a loan can meet the repayments and if there is any direct impact on the ability to repay in relation to their current status :marital status, employment, number of children, etc'.

This way the bank’s loan division can rank the customers "Credit Score" and minimize the risk when they approve request.


<b style="font-size: 120%">Index:</b>

<a href='#one'>I   - Loading Dataset</a>

<a href='#two'>II  - Data exploration</a>

<a href='#three'>III - Data PreProcessing missing values</a>

<a href='#four'>IV - Categorization of data</a>

<a href='#five'>V  - Checking the Hypotheses</a>

<a href='#six'>VI  - General Conclusion</a>

<b style="font-size: 120%">The connection between taking a loan and:</b>

<a href='#child'>Children</a>

<a href='#marital'>Marital status</a>

<a href='#income'>income</a>

<a href='#purposes'>purposes</a>

<b><a id="one" style="font-size: 140%; text-decoration: underline;">I - Loading Dataset</a></b>

In [1]:
# import pandas, numpay and nltk liberaries
import pandas as pd
import numpy as np
import nltk
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from collections import Counter
english_stemmer = SnowballStemmer('english')
from nltk.stem import WordNetLemmatizer
wordnet_lemma = WordNetLemmatizer() 
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
is_noun = lambda pos: pos[:2] == 'NN'
nltk.download('averaged_perceptron_tagger')

# import Interactivesheel helper
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#Load dataset
credit_score = pd.read_csv('/datasets/credit_scoring_eng.csv')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!




<b><a id="two" style="font-size: 140%; text-decoration: underline;">II - Data exploration</a></b>



In [2]:
# first look at the dataset information : rows , columns, missing values etc' 
# use many options output to investigate the data

pd.concat([credit_score.dtypes, credit_score.count(),credit_score.isna().sum(),credit_score.isna().sum()/len(credit_score)], keys=['type','count','na','na%'], axis=1)
credit_score.head(5)
credit_score.tail(5)
credit_score.sample(5)
credit_score.describe()
credit_score.describe(include='all')

,type,count,na,na%
children,int64,21525,0,0.000000
days_employed,float64,19351,2174,0.100999
dob_years,int64,21525,0,0.000000
education,object,21525,0,0.000000
education_id,int64,21525,0,0.000000
family_status,object,21525,0,0.000000
family_status_id,int64,21525,0,0.000000
gender,object,21525,0,0.000000
income_type,object,21525,0,0.000000
debt,int64,21525,0,0.000000


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car
21524,2,-1984.507589,40,secondary education,1,married,0,F,employee,0,13127.587,to buy a car


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
1904,0,352241.585550,59,bachelor's degree,0,married,0,F,retiree,0,16944.861,transactions with commercial real estate
20691,0,NaN,47,secondary education,1,divorced,3,F,business,0,NaN,to get a supplementary education
16269,0,-828.981819,47,secondary education,1,married,0,F,business,0,21721.115,university education
2853,0,-4989.068202,64,secondary education,1,married,0,M,business,0,39627.089,purchase of the house
7133,0,360995.861443,63,secondary education,1,married,0,M,retiree,0,17950.737,to own a car


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
count,21525.000000,19351.000000,21525.000000,21525,21525.000000,21525,21525.000000,21525,21525,21525.000000,19351.000000,21525
unique,NaN,NaN,NaN,15,NaN,5,NaN,3,8,NaN,NaN,38
top,NaN,NaN,NaN,secondary education,NaN,married,NaN,F,employee,NaN,NaN,wedding ceremony
freq,NaN,NaN,NaN,13750,NaN,12380,NaN,14236,11119,NaN,NaN,797
mean,0.538908,63046.497661,43.293380,NaN,0.817236,NaN,0.972544,NaN,NaN,0.080883,26787.568355,NaN
std,1.381587,140827.311974,12.574584,NaN,0.548138,NaN,1.420324,NaN,NaN,0.272661,16475.450632,NaN
min,-1.000000,-18388.949901,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN,0.000000,3306.762000,NaN
25%,0.000000,-2747.423625,33.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,16488.504500,NaN
50%,0.000000,-1203.369529,42.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,23202.870000,NaN
75%,1.000000,-291.095954,53.000000,NaN,1.000000,NaN,1.000000,NaN,NaN,0.000000,32549.611000,NaN


<b style="font-size: 120%">Checking 0's</b>

In [3]:
for i in credit_score:
    print(i,len(credit_score[credit_score[i]==0]))

children 14149
days_employed 0
dob_years 101
education 0
education_id 5260
family_status 0
family_status_id 12380
gender 0
income_type 0
debt 19784
total_income 0
purpose 0



`'dob_years'`: 101 rows of value 0 , probably ty·po since no one will give a loan to born baby, you need to be at least 18 year of age

`'children'`:  14149 rows of value 0, which is normal since not all the customers must have children

`'family_status_id'`: 12380 rows of value 0, since this column share data with the column `'family_status'` we can marged them

`'education_id'`: 12380 rows of value 0, since this column share data with the column `'education'` we can marged them


<b style="font-size: 120%">Dataset Info:</b>

In [4]:
credit_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


<b style="font-size: 120%">contains : 21,525 rows x 12 Columns</b>

<b style="font-size: 120%">Our Columns:</b> 

- `children` - the number of children in the family

- `days_employed` - work experience in days

- `dob_years` - client's age in years

- `education` - client's education

- `education_id` - education identifier

- `family_status` - marital status

- `family_status_id` - marital status identifier

- `gender` - gender of the client

- `income_type` - type of employment

- `debt` - was there any debt on loan repayment

- `total_income` - monthly income

- `purpose` - the purpose of obtaining a loan

<b style="font-size: 120%">Missing values:</b> 

`'days_employed'`: only 19351 of 21525 rows (diffrence of 2174 rows)

`'total_income'` : only 19351 of 21525 rows (diffrence of 2174 rows)

<b style="font-size: 120%">Type:</b> 

`'days_employed'`: Float when expected to Integer

`'total_income'` : Float when expected to Integer

<b style="font-size: 120%">Invalid Values (numeric columns) :</b> 

`'children'`: -1 & 20 in the numbers of children in the famliy probably ty·po

`'days_employed'`: negative values in column that should be present positive values (the highest number is 401,755 which is around 1,100 years 

`'dob_years'`: already explain in Checking 0's


<b style="font-size: 120%"> Categorical columns:</b> 

`'education'`:duplicate values using case sensitivity ex':"Secondary Education","SECONDARY EDUCATION") 

`'purpose'`:duplicate values for the same purpose ex':"wedding ceremony", "to have a wedding" insted of just wedding

<b><a id="three" style="font-size: 140%; text-decoration: underline;">III - Data PreProcessing missing values</a></b>

Let's focus on the columns that have missing values and Checking each individually:

In [5]:
# loop throu the columns with the missing values
for m in credit_score:
    if credit_score[m].isnull().sum() > 0:
        print(m)

days_employed
total_income


<b style="font-size: 100%"> Process the missing values data of: `'days_employed'` column</b> 

In [6]:
# Let's look at the filtered table with missing values in the the first column with missing data
credit_score[credit_score['days_employed'].isnull()]
credit_score[credit_score['days_employed'].notna()]
credit_score['days_employed'].nunique()
credit_score['days_employed'].value_counts()/len(credit_score)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


19351

-986.927316     0.000046
-7026.359174    0.000046
-4236.274243    0.000046
-6620.396473    0.000046
-1238.560080    0.000046
                  ...   
-2849.351119    0.000046
-5619.328204    0.000046
-448.829898     0.000046
-1687.038672    0.000046
-582.538413     0.000046
Name: days_employed, Length: 19351, dtype: float64

In [7]:
credit_score[(credit_score.days_employed.isnull())&(credit_score.total_income.isnull())].count()

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

**Intermediate conclusion**

As we can see there is 2174 rows of missing values in both columns`'days_employed'` &`'total_income'`, which mean the missing values are symmetric and around 10% of the dataset, let continue our investigate with other columns to see if there is any connection with other values for specific client characteristic.

let's continue to `'education_id'`

In [8]:
# ratio 'days_employed' that notNA with 'education_id'
credit_score[credit_score.days_employed.notna()]['education_id'].value_counts()/(len(credit_score)-2174)

1    0.707612
0    0.243708
2    0.034882
3    0.013488
4    0.000310
Name: education_id, dtype: float64

In [9]:
# ratio of days_employed that IsNA with 'education_id'
credit_score[credit_score.days_employed.isna()]['education_id'].value_counts()/2174

1    0.708372
0    0.250230
2    0.031739
3    0.009660
Name: education_id, dtype: float64

In [10]:
# ratio of education_id
credit_score['education_id'].value_counts()/len(credit_score)

1    0.707689
0    0.244367
2    0.034564
3    0.013101
4    0.000279
Name: education_id, dtype: float64

**Intermediate conclusion**

after checking the ratio in the values of column `'education_id'`, which show the same result in all the test we made , we come to conclusion that there is no relationship between the missing values of column `'days_employed'` and column `'education_id'`

let's continue to `'family_status_id'`

In [11]:
# ratio 'days_employed' that notNA with family_status_id
credit_score[credit_score.days_employed.notna()]['family_status_id'].value_counts()/(len(credit_score)-2174)

0    0.575836
1    0.193013
4    0.130484
3    0.055966
2    0.044701
Name: family_status_id, dtype: float64

In [12]:
# ratio of days_employed that IsNA with 'family_status_id'
credit_score[credit_score.days_employed.isna()]['family_status_id'].value_counts()/2174

0    0.568997
1    0.203312
4    0.132475
3    0.051518
2    0.043698
Name: family_status_id, dtype: float64

In [13]:
# ratio of family_status_id
credit_score['family_status_id'].value_counts()/len(credit_score)

0    0.575145
1    0.194053
4    0.130685
3    0.055517
2    0.044599
Name: family_status_id, dtype: float64

**Intermediate conclusion**

like the `'education_id'` we can see the same result also in the column `'family_status_id'`, which bring us to conclusion that the values of of column `'days_employed'`are random and there isn't any connection between tham to other columns / characteristic of the client.

<b style="font-size: 100%"> Restoring missing values in: `'days_employed'` column</b> 

Since we already know that there is about 10% of missing value in our dataset , that come from the negative number that show in the `'days_employed'`and represent more than 1k years , i think it's time to check if there is option to bring this to more logical numbers maybe years , months or even hours, let dig in

In [14]:
credit_score['days_employed'] = credit_score['days_employed'].abs()
credit_score['Month_work'] = credit_score.days_employed.abs()/365/24*12
credit_score['Month_work'] = credit_score['Month_work'].abs().round()

# check the info about the new column
credit_score.Month_work.describe()/12
credit_score.Month_work.value_counts()
credit_score.Month_work.isna().value_counts()

count    1612.583333
mean        7.637525
std        15.871560
min         0.000000
25%         0.083333
50%         0.250000
75%         0.666667
max        45.833333
Name: Month_work, dtype: float64

1.0     3901
2.0     2910
3.0     1989
0.0     1827
4.0     1558
        ... 
21.0       5
22.0       5
24.0       1
23.0       1
25.0       1
Name: Month_work, Length: 127, dtype: int64

False    19351
True      2174
Name: Month_work, dtype: int64

We can certainly see that the huge negative value been turned into a positive one and not only that when we divide it by 365 days a year into 24 hours multiplied by 12 months, it makes a lot more sense of 45 years by Max.

there is still gap between that show 7 years in Mean to 3 months in median , so let fill up this gap.

In [15]:
# fill the missing values with the median 
credit_score['Month_work']=credit_score['Month_work'].fillna(credit_score.Month_work.median())

In [16]:
# check the month_employed after the operation
credit_score.Month_work.describe()
credit_score.Month_work.isna().value_counts()
credit_score.Month_work.value_counts()

count    21525.000000
mean        82.696725
std        182.549366
min          0.000000
25%          1.000000
50%          3.000000
75%          7.000000
max        550.000000
Name: Month_work, dtype: float64

False    21525
Name: Month_work, dtype: int64

3.0     4163
1.0     3901
2.0     2910
0.0     1827
4.0     1558
        ... 
21.0       5
22.0       5
24.0       1
23.0       1
25.0       1
Name: Month_work, Length: 127, dtype: int64

**Intermediate conclusion**

after fillna the missing values we can see that about 75% of the customers been work less than 7 months.

<b style="font-size: 100%"> Process the missing values data of: `'gender'` column</b> 

In [17]:
# Let's see the values in the column
credit_score['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [18]:
# Address the problematic values, if they exist
credit_score['gender'].replace(['XNA'],['F'], inplace=True)

In [19]:
# Check the result - make sure it's fixed
credit_score['gender'].value_counts()

F    14237
M     7288
Name: gender, dtype: int64

**Intermediate conclusion**

After looking at the results of the table i decied to go and replace or move the 'XNA' to the `'gender'` that have higher percentage in the dataset which is is female

<b style="font-size: 100%"> Process the missing values data of: `'total_income'` column</b> 

In [20]:
credit_score.total_income.describe()
credit_score.total_income.isna().sum()

count     19351.000000
mean      26787.568355
std       16475.450632
min        3306.762000
25%       16488.504500
50%       23202.870000
75%       32549.611000
max      362496.645000
Name: total_income, dtype: float64

2174

**Intermediate conclusion**

as we see before the total missing values in `'total_income'` is 2174 rows , there isn't a big gap between the result of mean & median in this case let's check it with other columns.

In [21]:
# check with income_type & gender:
credit_score.groupby('income_type')['total_income'].median()
credit_score.groupby('gender')['total_income'].median()

income_type
business                       27577.2720
civil servant                  24071.6695
employee                       22815.1035
entrepreneur                   79866.1030
paternity / maternity leave     8612.6610
retiree                        18962.3180
student                        15712.2600
unemployed                     21014.3605
Name: total_income, dtype: float64

gender
F    21465.165
M    26834.295
Name: total_income, dtype: float64

**Intermediate conclusion**

As we can see above when we check the column `'total_income'` with `'income_type'` & `'gender'` there is a diffrence in the values

<b style="font-size: 100%"> Restoring missing values in: `'total_income'` column</b>

to fix it, we will fillna it with .groupby avrage using the transform function 

In [22]:
#  Write a function that we will use for filling in missing values
credit_score['total_income'] = credit_score.groupby('income_type')['total_income']\
.transform(lambda grp : grp.fillna(np.mean(grp)))
credit_score.total_income.describe()

count     21525.000000
mean      26783.346520
std       15665.112650
min        3306.762000
25%       17247.708000
50%       24309.013000
75%       32386.793835
max      362496.645000
Name: total_income, dtype: float64

<b style="font-size: 100%"> Process the missing values data of: `'children'` column</b> 

In [23]:
credit_score.children.unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**Intermediate conclusion**

Here we dealing with probably ty·po as i said earlier since "-1" of `'children'`is something not reasonable and "20" isn't something so common which better to change to lower value

In [24]:
# [fix the data based on your decision]
credit_score['children'].replace([-1, 20],[1, 2], inplace=True)

In [25]:
# Checking the `children` column again to make sure it's all fixed
credit_score.children.unique()

array([1, 0, 3, 2, 4, 5])

**Intermediate conclusion**

We resolved this two issues by replace the value of "-1" to "1" and "20" to "2" , this didn't change the mean so much

<b style="font-size: 100%"> Process the missing values data of: `'education'` column</b> 

In [26]:
credit_score['education'].value_counts()

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
GRADUATE DEGREE            1
Graduate Degree            1
Name: education, dtype: int64

**Intermediate conclusion**

Here we have a problem of sensitivity capital letters as well as the same certificate with a slightly different name, which can be caused by a free text area in the submission form or another system that we import from into our data set

In [27]:
credit_score['education'] = credit_score['education'].str.lower()
credit_score.loc[credit_score['education']=='graduate degree', 'education']="bachelor's degree"
credit_score['education'].value_counts()

secondary education    15233
bachelor's degree       5266
some college             744
primary education        282
Name: education, dtype: int64

**Intermediate conclusion**

We no longer have any issue in the column

In [28]:
# check the connection between education & education_id columns
credit_score['education'].value_counts()
credit_score.pivot_table(index = 'education_id', columns='education',values = 'debt', aggfunc = ['count'],)

secondary education    15233
bachelor's degree       5266
some college             744
primary education        282
Name: education, dtype: int64

count                                        \
education    bachelor's degree primary education secondary education   
education_id                                                           
0                       5260.0               NaN                 NaN   
1                          NaN               NaN             15233.0   
2                          NaN               NaN                 NaN   
3                          NaN             282.0                 NaN   
4                          6.0               NaN                 NaN   

                           
education    some college  
education_id               
0                     NaN  
1                     NaN  
2                   744.0  
3                     NaN  
4                     NaN

In [29]:
# since we change the 'education' column, we need to do same for 'education_id'
credit_score.loc[credit_score['education_id']==4, 'education_id']=0
credit_score['education'].value_counts()

secondary education    15233
bachelor's degree       5266
some college             744
primary education        282
Name: education, dtype: int64

**Intermediate conclusion**

We no longer have any issue in the `'education_id'` column and which also bring us to conclusion that there is obvious connection between `'education'` & `'education_id'`

<b style="font-size: 100%"> Process the missing values data of: `'family_status'` column</b> 

In [30]:
# checking 'family_status' and if there is connection to 'family_status_id'
credit_score['family_status'].value_counts()
credit_score.pivot_table(index = 'family_status_id', columns='family_status',values = 'debt', aggfunc = ['count'],)

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

count                                            
family_status    civil partnership divorced  married unmarried widow / widower
family_status_id                                                              
0                              NaN      NaN  12380.0       NaN             NaN
1                           4177.0      NaN      NaN       NaN             NaN
2                              NaN      NaN      NaN       NaN           960.0
3                              NaN   1195.0      NaN       NaN             NaN
4                              NaN      NaN      NaN    2813.0             NaN

In [31]:
#checling the family_status_id values
credit_score['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

**Intermediate conclusion**

We can see there isn't any issue in this two columns , but definitely they connected to each other

<b style="font-size: 100%"> Process the missing values data of: `'income_type'` column</b> 

In [32]:
#checling the income_type values
credit_score['income_type'].value_counts()

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
entrepreneur                       2
unemployed                         2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

**Intermediate conclusion**

We can't find any issue with `'income_type'` column

<b style="font-size: 100%"> Process the missing values data of: `'debt'` column</b> 

In [33]:
#checling the debt values
credit_score['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

**Intermediate conclusion**

since we turn the missing value of `'days_employed'` in to "Month" (in the new column `'Month_work'`) and all the cloumns show now 21525 rows of values so we don't really need this column anymore in our dataset and can be remove if we need

<b style="font-size: 100%"> Process the missing values data of: `'dob_years'` column</b> 

In [34]:
credit_score['dob_years'].isna().sum()
credit_score[credit_score.dob_years==0]['dob_years'].value_counts()
credit_score['dob_years'].describe()

0

0    101
Name: dob_years, dtype: int64

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

**Intermediate conclusion**

There isn't any Nan values in `'dob_years'`, the only issue we found is 101 rows that show 0's, which isn't an age for our customer that request loan , let's change it to our avarge age (43) using the mean function.

In [35]:
# Address the issues in the `dob_years` column, if they exist
credit_score['dob_years']=credit_score['dob_years'].replace(0,credit_score['dob_years'].mean().round())

In [36]:
# Checking the `dob_years` column again to make sure it's all fixed
credit_score['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75])

**Conclusions**

in the way we find issues that our dataset show columns without any issue and others that need some tweaks to fix them using diffrence method function like replace, mean/median , transform etc')

so let's check the hard work and see the result 

In [37]:
credit_score.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income,Month_work
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000
mean,0.479721,66914.728907,43.495145,0.816121,0.972544,0.080883,26783.346520,82.696725
std,0.755528,139030.880527,12.218213,0.545726,1.420324,0.272661,15665.112650,182.549366
min,0.000000,24.141633,19.000000,0.000000,0.000000,0.000000,3306.762000,0.000000
25%,0.000000,927.009265,34.000000,1.000000,0.000000,0.000000,17247.708000,1.000000
50%,0.000000,2194.220567,43.000000,1.000000,0.000000,0.000000,24309.013000,3.000000
75%,1.000000,5537.882441,53.000000,1.000000,1.000000,0.000000,32386.793835,7.000000
max,5.000000,401755.400475,75.000000,3.000000,4.000000,1.000000,362496.645000,550.000000


<b style="font-size: 100%">Checking for Duplicates</b> 

In [38]:
# Checking duplicates
credit_score[credit_score.duplicated()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,Month_work
2849,0,NaN,41,secondary education,1,married,0,F,employee,0,25820.841683,purchase of the house for my family,3.0
3290,0,NaN,58,secondary education,1,civil partnership,1,F,retiree,0,21940.394503,to have a wedding,3.0
4182,1,NaN,34,bachelor's degree,0,civil partnership,1,F,employee,0,25820.841683,wedding ceremony,3.0
4851,0,NaN,60,secondary education,1,civil partnership,1,F,retiree,0,21940.394503,wedding ceremony,3.0
5557,0,NaN,58,secondary education,1,civil partnership,1,F,retiree,0,21940.394503,to have a wedding,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,NaN,64,secondary education,1,married,0,F,retiree,0,21940.394503,supplementary education,3.0
21032,0,NaN,60,secondary education,1,married,0,F,retiree,0,21940.394503,to become educated,3.0
21132,0,NaN,47,secondary education,1,married,0,F,employee,0,25820.841683,housing renovation,3.0
21281,1,NaN,30,bachelor's degree,0,married,0,F,employee,0,25820.841683,buy commercial real estate,3.0


In [39]:
# Address the duplicates, if they exist
credit_score.drop_duplicates(inplace=True)

In [40]:
# Last check whether we have any duplicates
credit_score.duplicated().sum()

0

In [41]:
# Check the size of the dataset that you now have after your first manipulations with it
credit_score.shape

(21454, 13)

**Intermediate conclusion**

there was 71 row of duplicated data , that now been removed from our dataset so our total rows after this processes is 21454 instead of 21525 at the beginning 

<b><a id="four" style="font-size: 140%; text-decoration: underline;">IV - Categorization data</a></b>

<b style="font-size: 100%"> Categorization data of: `'total_income'` column</b> 

In [42]:
# Checking total_income column
credit_score['total_income'].describe()

count     21454.000000
mean      26789.131104
std       15689.696250
min        3306.762000
25%       17219.817250
50%       24302.006000
75%       32386.793835
max      362496.645000
Name: total_income, dtype: float64

**Intermediate conclusion**

As we can see everything looks fine in the numbers of "`total income`", because we are talking about an estimate of total income, we need a category from the calendar world, also known as quarters, that will help us present our income better.   

In [43]:
def income_by_quarters(income_quarters) :
    if income_quarters > 0 and income_quarters <= 17000:
        return 'First quarter'
    elif income_quarters > 17000 and income_quarters <= 25000 :
        return 'Second quarter'
    elif income_quarters > 25000 and income_quarters <= 35000:
        return 'Third quarter'
    elif income_quarters > 35000 and income_quarters < 370000:
        return 'Fourth quarter '
credit_score['income_by_quarters'] = credit_score['total_income'].apply(income_by_quarters)
credit_score['income_by_quarters'].head(10)

0    Fourth quarter 
1     Second quarter
2     Second quarter
3    Fourth quarter 
4      Third quarter
5    Fourth quarter 
6    Fourth quarter 
7     Second quarter
8      First quarter
9     Second quarter
Name: income_by_quarters, dtype: object

<b style="font-size: 100%"> Categorization data of: `'purpose'` column</b> 

In [44]:
# Checking purpose column
target = credit_score['purpose'].unique()
display(target)
len(target)

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

38

**Intermediate conclusion**

in this category the dataset show us 38 unique values, but closer look already show repeating value with the same target in different ways , we need to lemmatize it so the `'purpose'` column values will show the main one like : House , car , education and so on 

In [45]:
lemmas = []
for text in target:
    words = nltk.word_tokenize(text.lower())
    lemmas += [wordnet_lemma.lemmatize(w, pos='n') for (w,pos) in nltk.pos_tag(words) if w not in stopwords and is_noun(pos)]
print(lemmas)

['purchase', 'house', 'car', 'purchase', 'education', 'wedding', 'housing', 'transaction', 'education', 'wedding', 'purchase', 'house', 'family', 'estate', 'estate', 'estate', 'construction', 'property', 'property', 'property', 'second-hand', 'car', 'car', 'transaction', 'estate', 'estate', 'housing', 'transaction', 'estate', 'car', 'second-hand', 'car', 'purchase', 'education', 'car', 'wedding', 'ceremony', 'education', 'purchase', 'house', 'estate', 'transaction', 'education', 'car', 'purchase', 'car', 'profile', 'education', 'university', 'education', 'property', 'car', 'housing', 'renovation', 'university']


In [46]:
purpose_categories = list(Counter(lemmas).keys())
category_purpose = {text_purpose:next(category for category in purpose_categories if english_stemmer.stem(category) in text_purpose or category in text_purpose) for text_purpose in set(credit_score['purpose'])}
category_purpose

{'education': 'education',
 'to get a supplementary education': 'education',
 'purchase of my own house': 'purchase',
 'housing renovation': 'house',
 'purchase of a car': 'purchase',
 'housing': 'house',
 'building a real estate': 'estate',
 'housing transactions': 'house',
 'getting an education': 'education',
 'car': 'car',
 'having a wedding': 'wedding',
 'wedding ceremony': 'wedding',
 'supplementary education': 'education',
 'purchase of the house for my family': 'purchase',
 'transactions with my real estate': 'transaction',
 'second-hand car purchase': 'purchase',
 'building a property': 'property',
 'buy real estate': 'estate',
 'buying my own car': 'car',
 'car purchase': 'purchase',
 'getting higher education': 'education',
 'to have a wedding': 'wedding',
 'university education': 'education',
 'going to university': 'university',
 'cars': 'car',
 'construction of own property': 'construction',
 'to buy a car': 'car',
 'to become educated': 'education',
 'transactions with c

**Intermediate conclusion**

as i said before and we see it again there is kind of dupliceted values in `'purpose'`, let fix it 

In [47]:
for key, value in category_purpose.items():
    if not value=='car' and 'car' in key:
        category_purpose[key] = 'car'
        continue
    if not value=='house' and ('house' in key or 'estate' in key):
        category_purpose[key] = 'house'
        continue
    if not value=='property' and ('construction' in key or 'property' in key):
        category_purpose[key] = 'property'
        continue
category_purpose

{'education': 'education',
 'to get a supplementary education': 'education',
 'purchase of my own house': 'house',
 'housing renovation': 'house',
 'purchase of a car': 'car',
 'housing': 'house',
 'building a real estate': 'house',
 'housing transactions': 'house',
 'getting an education': 'education',
 'car': 'car',
 'having a wedding': 'wedding',
 'wedding ceremony': 'wedding',
 'supplementary education': 'education',
 'purchase of the house for my family': 'house',
 'transactions with my real estate': 'house',
 'second-hand car purchase': 'car',
 'building a property': 'property',
 'buy real estate': 'house',
 'buying my own car': 'car',
 'car purchase': 'car',
 'getting higher education': 'education',
 'to have a wedding': 'wedding',
 'university education': 'education',
 'going to university': 'university',
 'cars': 'car',
 'construction of own property': 'property',
 'to buy a car': 'car',
 'to become educated': 'education',
 'transactions with commercial real estate': 'house',


**Intermediate conclusion**

As you can see we redefined the givers and gave each one a short and accurate name of the target, now that we see what it looks like in the column itself

In [48]:
credit_score['purpose'] = credit_score['purpose'].replace(category_purpose)
credit_score['purpose'].value_counts()

house         8273
car           4306
education     3517
property      2538
wedding       2324
university     496
Name: purpose, dtype: int64

**Intermediate conclusion**

Now it's looks much better and `'purpose'` have short understanding value 

In [49]:
credit_score.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,Month_work,income_by_quarters
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,house,12.0,Fourth quarter
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car,6.0,Second quarter
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,house,8.0,Second quarter
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,education,6.0,Fourth quarter
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,466.0,Third quarter
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,house,1.0,Fourth quarter
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,house,4.0,Fourth quarter
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,0.0,Second quarter
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,wedding,9.0,First quarter
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,house,3.0,Second quarter


<b>** Short Dictionary**</b>

<b>House</b> : building for human habitation, especially one that is lived in by a family

<b>property</b> : belongs to a person but he or she or one of is family members do not live in it and is usually for business needs such as an apartment, building or office for rent, etc.

<b>education</b> : process of receiving or giving systematic instruction, especially at a school or university

<b>car</b> : a four-wheeled road vehicle that is powered by an engine or electric motor and is able to carry a small number of people.

<b>wedding</b> : a marriage ceremony, especially considered as including the associated celebrations

<b><a id="five" style="font-size: 140%; text-decoration: underline;">V - Checking the Hypotheses</a></b>

**Is there a correlation between having children and paying back on time?**
<a id='child'></a>

In [50]:
print('average debt defaul rate is {:.1%}'.format(credit_score[credit_score.debt == 1].debt.count()/len(credit_score)))
print('average debt defaul rate among no-children is {:.1%}'.format(credit_score[(credit_score.debt==1)&(credit_score.children==0)].debt.count()/credit_score[credit_score.children==0].debt.count()))
print('average debt defaul rate among children owners is {:.1%}'.format(credit_score[(credit_score.debt==1)&(credit_score.children>0)].debt.count()/credit_score[credit_score.children>0].debt.count()))
print()
print('------------------ defaul rate by children ------------------------------------')
print()
for i in credit_score.children.unique():
    print('average debt defaul rate among',i,' children is {:.1%}'.format(credit_score[(credit_score.debt==1)&(credit_score.children==i)].debt.count()/credit_score[credit_score.children==i].debt.count()))

average debt defaul rate is 8.1%
average debt defaul rate among no-children is 7.5%
average debt defaul rate among children owners is 9.2%

------------------ defaul rate by children ------------------------------------

average debt defaul rate among 1  children is 9.2%
average debt defaul rate among 0  children is 7.5%
average debt defaul rate among 3  children is 8.2%
average debt defaul rate among 2  children is 9.5%
average debt defaul rate among 4  children is 9.8%
average debt defaul rate among 5  children is 0.0%


**Conclusion**

We can see that defaulted customer without Children is little bit lower than  customer with children (7.5% compere to 9.2%) 

**Is there a correlation between marital status and paying back on time?**
<a id='marital'></a>

In [51]:
for i in credit_score.family_status.unique():
    print('average debt defaul rate among',i,' is {:.1%}'.format(credit_score[(credit_score.debt==1)&(credit_score.family_status==i)].debt.count()/credit_score[credit_score.family_status==i].debt.count()))

average debt defaul rate among married  is 7.5%
average debt defaul rate among civil partnership  is 9.3%
average debt defaul rate among widow / widower  is 6.6%
average debt defaul rate among divorced  is 7.1%
average debt defaul rate among unmarried  is 9.8%


**Conclusion**

Here we can see that defaulted unmarried is higher and stand on 9.7% compere to the lowest defaulted widow that show 6.6% ,this can be due to several reasons: inheritance or money left by the deceased, waiver or cut by the creditors, etc'. becuase of this statistical i don't think there is huge diffrent between them 

**Is there a correlation between income level and paying back on time?**
<a id='income'></a>

In [52]:
for i in credit_score.income_by_quarters.unique():
    print('average debt defaul rate among',i,' is {:.1%}'.format(credit_score[(credit_score.debt==1)&(credit_score.income_by_quarters==i)].debt.count()/credit_score[credit_score.income_by_quarters==i].debt.count()))

average debt defaul rate among Fourth quarter   is 7.1%
average debt defaul rate among Second quarter  is 8.8%
average debt defaul rate among Third quarter  is 8.3%
average debt defaul rate among First quarter  is 7.8%


**Conclusion**

As we can see there isn't much difference between the groups when the low is 7.1% and the higher is 8.8%

**How does credit purpose affect the default rate?**
<a id='purposes'></a>

In [53]:
for i in credit_score.purpose.unique():
    print('average debt defaul rate among',i,' is {:.1%}'.format(credit_score[(credit_score.debt==1)&(credit_score.purpose==i)].debt.count()/credit_score[credit_score.purpose==i].debt.count()))

average debt defaul rate among house  is 7.2%
average debt defaul rate among car  is 9.4%
average debt defaul rate among education  is 9.3%
average debt defaul rate among wedding  is 8.0%
average debt defaul rate among property  is 7.5%
average debt defaul rate among university  is 8.7%


**Conclusion**

If we talked about the same purpose could be diffrent for the bank to take a risk to loaner, but since it's not the case here and we can't compere buying a house to go and get education the number between the lower 7.1% to the highest 9.3%, not show such a differences.

<b><a id="six" style="font-size: 170%; text-decoration: underline;">VI - General Conclusion</a></b>

<b style="font-size: 110%; text-decoration: underline;">INTRODUCTION</b>

Our goal is to analysis and determine for bank’s loan division about is  customers depend on different categories: children , income , family status , education etc.', which customer can pay the loan and how will be difficulty for how to pay over time.

This as long as everything is normal, since we living in a dynamic world and thing's can be change without notice and impact the ability to pay loans.

The Dataset contains the following info columns:

- `children` - the number of children in the family


- `days_employed` - work experience in days


- `dob_years` - client's age in years


- `education` - client's education


- `education_id` - education identifier


- `family_status` - marital status


- `family_status_id` - marital status identifier


- `gender` - gender of the client


- `income_type` - type of employment


- `debt` - was there any debt on loan repayment


- `total_income` - monthly income


- `purpose` - the purpose of obtaining a loan


<b style="font-size: 110%; text-decoration: underline;">PREPROCESSING - Missing values</b>


as soon as we start explorer or dataset we notice that there are two columns with issues aka missing values `days_employed` and` total_income` both of them miss 2174 rows from the total 21525 entries which is about 10% of all the data.

i start to investigate and i found not only negative values in `days_employed` columns but also if you calculated it's about 1k of years of missing values very high. this can happens in case of entering wrong numbers or import it from other system that do the math differently other way i deiced to see if there is option to fix it or at least make it more reasonable, so first i change the numbers to positive and then we still see the big number less the "-" beside it so divide it by 365 days divide by 24 hours multiply it by 12 this turn it to Normal numbers in high resolution. than i check it with other columns and found there is no connection between this column and the other ones so we can not point on specific client characteristic and i fillna the missing values so now we have no missing rows.

i also found the same number of missing rows in the `total_income` important column without a doubt, i check this with other columns and did not find any connection that can lead to this issue other than this value not enter or import from other info system, so i deiced also here to fillna but with the income_type using the transform function and mean for average so now this columns is full and there is no missing value.

also been fixed little issues that i found in other columns like number of `children` in the family show ty · po issue of" -1 "and" 20 "or uppercase in` education`

<b style = "font-size: 110%; text-decoration: underline;"> Categorizing Data </b>

first column that i Categorized is the `'total_income'` i did function that split the income into 4 quarters, to Memic the Calander method over the year it's also split the customers into groups.

`'Purpose'` here we find big issue that can cause by free text area so the customers can write and tell for what they want the loan, it's fine in paper but this big mass in dataset that show so many options to the same target , this need to be simple like: house, car etc.


<b style = "font-size: 110%; text-decoration: underline;"> income: </b>

the best customers the pay in time there a loan our the lowest income level after it the one with the highest income and in middle in last place.


<b style = "font-size: 110%; text-decoration: underline;"> Loan purposes: </b>
average debt defaul loan: 7.1% house, 9.3% car and education, 8.7% university, 7.9% wedding and 7.5% property

<b style="font-size: 110%; text-decoration: underline;">Overall conclusion</b>

So what did we learn in this project, that in order for the bank's credit department to be able to approve a loan, it needs to know data that will define for them the existing risks according to various parameters. In the report we received, it seemed that the first thing that stood out was the lack of information, some of which was very critical of the bank's decision on the subject and even in one column the numbers were negative and not very logical. The same topic but it was written in uppercase and lowercase and there were also fields in which they probably preferred not to identify, so that we could present the information more reliably we treated the data as much as possible and brought the table to a position where no fields are missing and information seems effective. Treating the same customers, as we have obtained statistical results that show what those customers are and their ability to repay in relation to other customers